# Import Libraries

In [1]:
import webbrowser
import glob
import pandas as pd
import time
import shutil, os
import numpy as np
import datetime
from datetime import date
################ WEB SCRAPING MODULES ############
import bs4
from fake_useragent import UserAgent
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings('ignore')

# Create Folders

In [2]:
today = date.today() 
try:
    os.mkdir(str(today))
    os.mkdir(str(today)+'/Patent files')
    os.mkdir(str(today)+'/Trials files')
except Exception:
    pass

# Get list of non-public companies

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # open Chrome driver/window
user=input('Do you want to enter companies one by one?(y/n): ')
l=[]
if(user=='y'):
    more='y'
    while(more=='y'):
        a=input('enter company name: ')
        l.append(a)
        more=input('do you want to enter more companies(y/n): ')
else:
    csv=input('enter name of excel file: ') # put non-public.xlsx
    companies=input('enter column containing list of companies: ') # put Company
    try:
        l=list(pd.read_csv(csv)[companies])
    except:
        l=list(pd.read_excel(csv,engine='openpyxl')[companies])
l



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\aparg\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


Do you want to enter companies one by one?(y/n): n
enter name of excel file: non-public.xlsx
enter column containing list of companies: Company


['MolecuLight Inc.',
 'PicnicHealth',
 'Syapse',
 'Sapience Therapeutics',
 'Surgical Theater',
 'Onlume Surgical',
 'KAHR Medical',
 'CorWave SA',
 'Goldfinch Bio']

# Scrape [Google Patents](https://patents.google.com/) for Patents

## Sheet 1

## Delete old csv files in directories (if any)

In [4]:
def delete(directory):
    filelist = [ f for f in os.listdir(directory) if f.endswith(".csv")]
    for f in filelist:
        os.remove(os.path.join(directory, f))

In [5]:
# from "Downloads" directory
delete('C:/Users/aparg/Downloads')

In [6]:
# from directory created
directory='C:/Users/aparg/Desktop/LifeSciences/'+str(today)
delete(directory)

In [7]:
# from subdirectory
delete(directory+'/Patent files')

## Scrape patents for each company

In [8]:
for j,i in enumerate(l):
    try:
        print(j,i)
        link="https://patents.google.com/xhr/query?url=assignee%3D{}%26oq%3D{}&exp=&download=true".format("%2B".join(i.split()),"%2B".join(i.split()))
        # download csv
        webbrowser.open(link)
        # wait for 20 sec
        time.sleep(20)
        # read csv from downloads
        all_files = glob.glob(os.path.join(r'C:\Users\aparg\Downloads', "*.csv"))
        # rename csv
        os.rename(all_files[0],'C:/Users/aparg/Downloads/'+i+'.csv')
        # copy csv to req. folder
        shutil.copy('C:/Users/aparg/Downloads/'+i+'.csv', directory+'/Patent files')
        # delete csv from downloads
        delete('C:/Users/aparg/Downloads')
    except:
        pass

0 MolecuLight Inc.
1 PicnicHealth
2 Syapse
3 Sapience Therapeutics
4 Surgical Theater
5 Onlume Surgical
6 KAHR Medical
7 CorWave SA
8 Goldfinch Bio


## Combine all patent files

In [9]:
# read all csv files in subdirectory "Patent files"
all_files = glob.glob(os.path.join(directory+'/Patent files', "*.csv"))  
# concatenate all csv files present in "Patent files"
df_from_each_file = (pd.read_csv(f,header=1) for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)
concatenated_df=concatenated_df[['id','title','publication date','assignee','inventor/author']]

In [11]:
# filtering output based on "assignee"
appended_data = []
for company in l:
    s = pd.Series([str(i).lower() for i in concatenated_df['assignee']])
    k=s[s.str.contains(company.lower())].index
    if(list(k)!=[]):
        data = concatenated_df.loc[k]
        data['assignee']=[company for i in range(data.shape[0])]
        appended_data.append(data)
appended_data = pd.concat(appended_data)

In [12]:
# Feature Engineering "Year Filed" from "publication date"
appended_data['Year Filed']=appended_data['publication date'].apply(lambda x:str(x).split('-')[0])
# selecting required columns
appended_data=appended_data[['assignee','id', 'title', 'publication date','Year Filed','inventor/author']]
# renaming columns
appended_data.columns=['Company Name/Assignee','Patent No.','Title','Publication Date','Year Filed','Inventors/Authors']
appended_data['Year Filed']=appended_data['Year Filed'].replace('nan',np.nan)

In [13]:
appended_data.head()

,Company Name/Assignee,Patent No.,Title,Publication Date,Year Filed,Inventors/Authors
30,MolecuLight Inc.,AU-2019264864-A1,"Imaging drapes, packaging for drapes, methods ...",2021-01-07,2021,"Ralph DACOSTA, Danielle DUNHAM, Sonia GULIA, N..."
35,MolecuLight Inc.,US-D903863-S,Adapter for supporting a darkening drape,2020-12-01,2020,"Ralph S. DaCosta, Simon Treadwell, Sonia Gulia..."
36,MolecuLight Inc.,US-2020364862-A1,Wound imaging and analysis,2020-11-19,2020,"Ralph DACOSTA, Todd E. MEANEY, Todd Daynes, Ga..."
39,MolecuLight Inc.,WO-2020148726-A1,Système modulaire d'imagerie et d'analyse mult...,2020-07-23,2020,"Ralph DACOSTA, Simon Treadwell, Connor WRIGHT,..."
40,Sapience Therapeutics,US-10316077-B2,Cell-penetrating ATF5 polypeptides and uses th...,2019-06-11,2019,"Barry Jay Kappel, Jimmy Andrew Rotolo, Gene Me..."


In [14]:
appended_data.shape[0]

45

In [15]:
driver.close()

## Filter Scraped Patents using [USPTO](http://patft.uspto.gov/)

In [16]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # open Chrome driver/window
links=[]
for i in l:
    term="+".join(i.split())
    driver.get('http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=0&f=S&l=50&TERM1={}&FIELD1=ASNM&co1=AND&TERM2=&FIELD2=&d=PTXT'.format(term))
    source = driver.page_source
    soup=bs4.BeautifulSoup(source, 'html.parser')
    #################### GET LINKS OF PAGES FOR EACH COMPANY #####################
    try:
        no_of_pages=int(soup.find_all('strong')[len(soup.find_all('strong'))-1].text)
        for page in range(no_of_pages):
            link='http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r={}&f=G&l=50&co1=AND&d=PTXT&s1=%22{}%22.ASNM.&OS=AN/%22{}al%22&RS=AN/%22{}%22'.format(page+1,term,term,term)
            links.append(link)
    except:
        pass



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\aparg\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [17]:
title=[]
for j in links:
    driver.get(j)
    source = driver.page_source
    soup=bs4.BeautifulSoup(source, 'html.parser')
    try:
        text=soup.find('font',{'size':'+1'}).text.strip()
        title.append(text)
    except:
        pass
    time.sleep(5)

In [18]:
appended_data['Title']=appended_data['Title'].apply(lambda x:x.strip())
appended_data=appended_data.loc[appended_data['Title'].isin(list(set(title)))]

In [19]:
appended_data=appended_data.drop_duplicates(subset=['Company Name/Assignee', 'Title'])
driver.close()

# Scrape [Clinical Trials](https://clinicaltrials.gov/)

## Scrape trials for each company

In [20]:
for j,i in enumerate(l):
    try:
        print(j,i)
        link="https://clinicaltrials.gov/ct2/results/download_fields?cond=&term={}&type=&down_fmt=csv&down_flds=all&rslt=&age_v=&gndr=&intr=&titles=&outc=&spons=&lead=&id=&cntry=&state=&city=&dist=&locn=&fund=2&rsub=&strd_s=&strd_e=&prcd_s=&prcd_e=&sfpd_s=&sfpd_e=&rfpd_s=&rfpd_e=&lupd_s=&lupd_e=&sort=".format("+".join(i.split()))
        df = pd.read_csv(link)
        df=df[['First Posted','NCT Number','Title','Start Date','Completion Date','Sponsor/Collaborators']]
        df['Company Name']=[i for j in range(df.shape[0])]
        df.to_csv(directory+'/Trials files/'+i+'.csv',index=False)
    except:
        pass

0 MolecuLight Inc.
1 PicnicHealth
2 Syapse
3 Sapience Therapeutics
4 Surgical Theater
5 Onlume Surgical
6 KAHR Medical
7 CorWave SA
8 Goldfinch Bio


## Combine all trials files

In [21]:
all_files = glob.glob(os.path.join(directory+'/Trials files', "*.csv"))  
if(all_files!=[]):
    a=pd.read_csv(all_files[0])
    for i in all_files[1:]:
        a=pd.concat([a,pd.read_csv(i)])
        
    a['Principal Investigator']=['-' for j in range(a.shape[0])]
    a=a[['Company Name','NCT Number','Title', 'Start Date', 'Completion Date','Sponsor/Collaborators','Principal Investigator']] 
    a.columns=['Company Name','Clinical Trial Identifier','Title of study',
               'Start Date','End Date','Sponsor Company','Principal Investigator']
    a['C'] = a.apply(lambda x: str(x['Company Name']) in str(x['Sponsor Company']), axis=1)
    a=a.loc[a['C']==True]
    a=a.drop('C',axis=1)

In [22]:
if(all_files!=[]):
    ua = UserAgent()
    for i in range(a.shape[0]):
        url='https://clinicaltrials.gov/ct2/show/{}'.format(a.iloc[i,1])
        response = requests.get(url, {"User-Agent": ua.random})
        if response.status_code == 200:
            soup = bs4.BeautifulSoup(response.text, "html.parser")
            for j in soup.find_all('table',{'class':'ct-layout_table tr-indent2'}):
                m=j.text
                if('Study Director:' in m):
                    a.iloc[i,-1]=m[m.index('Study Director:')+16:]

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "c:\users\aparg\anaconda3\envs\divya\lib\site-packages\fake_useragent\utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "c:\users\aparg\anaconda3\envs\divya\lib\site-packages\fake_useragent\utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [47]:
a.head()

,Company Name,Clinical Trial Identifier,Title of study,Start Date,End Date,Sponsor Company,Principal Investigator
0,Goldfinch Bio,NCT04880291,First-In-Human Study of GFB-024 in Healthy Ove...,"May 5, 2021","November 6, 2021","Goldfinch Bio, Inc.",-
1,Goldfinch Bio,NCT04387448,A Study of TRPC5 Channel Inhibitor in Patients...,"July 28, 2020",December 2021,"Goldfinch Bio, Inc.",-
2,Goldfinch Bio,NCT04235621,A Study to Understand the Genetics and Clinica...,"December 20, 2019",June 2021,"Goldfinch Bio, Inc.",-
3,Goldfinch Bio,NCT03970122,FIH Ph 1 Study of TRPC5 Channel Inhibitor GFB-...,"May 29, 2019","April 4, 2020","Goldfinch Bio, Inc.",-
4,MolecuLight Inc.,NCT03540004,Evaluation of MolecuLight i:X as an Adjunctive...,"May 23, 2018","December 30, 2020","MolecuLight Inc.|SerenaGroup, Inc.",-


# Specify Date Range

In [23]:
try:
    d_range=input('Do you want for a date range(y/n): ')
    if(d_range=='y'):
        date1 = input('From(YYYY-MM-DD): ')
        date2 = input('To(YYYY-MM-DD): ')
        dates=[]
        dates2=[]
        start = datetime.datetime.strptime(date1, '%Y-%m-%d')
        end = datetime.datetime.strptime(date2, '%Y-%m-%d')
        step = datetime.timedelta(days=1)
    ################################################################
        while start <= end:
            dates.append (str(start.date()))
            start += step
        for i in dates:
            a1=i.split('-')
            x = datetime.datetime(int(a1[0]),int(a1[1]),int(a1[2]))
            dates2.append(x.strftime("%B %d, %Y"))
    ##################################################################
        appended_data=appended_data.loc[appended_data['Publication Date'].isin(dates)]
        a=a.loc[a['Start Date'].isin(dates2)]
except:
    pass

Do you want for a date range(y/n): n


 # Add not-scraped companies to DataFrame

In [24]:
# patents
not_scraped=list(set(l)-set(appended_data['Company Name/Assignee'].unique()))
for i in not_scraped:
    s2 = pd.DataFrame([i,np.nan,np.nan,np.nan,np.nan,np.nan]).T
    s2.columns=appended_data.columns
    appended_data=appended_data.append(s2)

In [25]:
# trials
if(all_files!=[]):
    not_scraped=list(set(l)-set(a['Company Name'].unique()))
    for i in not_scraped:
        s2 = pd.DataFrame([i,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]).T
        s2.columns=a.columns
        a=a.append(s2)

# Sheet 2

In [26]:
# patents
appended_data2=appended_data.groupby(['Company Name/Assignee']).size().reset_index(name='Value')
appended_data2['Patent (Y/N)']=['Y' for i in range(appended_data2.shape[0])]
for i in range(appended_data2.shape[0]):
    if(appended_data2['Company Name/Assignee'][i] in not_scraped):
        appended_data2['Value'][i]=0
        appended_data2['Patent (Y/N)'][i]='N'

In [27]:
# trials
appended_data3=a.groupby(['Company Name']).size().reset_index(name='Value')
appended_data3['Clinical Trials (Y/N)']=['Y' for i in range(appended_data3.shape[0])]
for i in range(appended_data3.shape[0]):
    if(appended_data3['Company Name'][i] in not_scraped):
        appended_data3['Value'][i]=0
        appended_data3['Clinical Trials (Y/N)'][i]='N'

# Save Outputs

In [30]:
# Write Sheet 1 and Sheet 2 as excel
name='/BiosectRx Patent Harvest-'+str(today)+'.xlsx'
writer = pd.ExcelWriter(directory+name, engine='xlsxwriter')
#appended_data.reset_index(inplace=True)
#appended_data2.reset_index(inplace=True)
appended_data2.to_excel(writer, sheet_name='Patent Check',index=False)
appended_data.to_excel(writer, sheet_name='Harvest',index=False)
writer.save()

In [31]:
name='/BiosectRx Trials Harvest-'+str(today)+'.xlsx'
writer = pd.ExcelWriter(directory+name, engine='xlsxwriter')
a.reset_index(inplace=True)
appended_data3.reset_index(inplace=True)
appended_data3.to_excel(writer, sheet_name='Trials Check',index=False)
a.to_excel(writer, sheet_name='Harvest',index=False)
writer.save()

# Populate in Template

In [32]:
# merge patents and trials
df3=pd.merge(appended_data, a, left_on='Company Name/Assignee', right_on='Company Name')
df3=df3.drop(['Company Name'],axis=1)
df3=df3.drop_duplicates()

In [ ]:
# open original template to get format
df=pd.read_excel('Template.xlsx')
df.columns=df.iloc[0,:]

In [34]:
df5=df3.copy()
df5=df5[['Company Name/Assignee', 'Patent No.', 'Title', 'Year Filed',
       'Inventors/Authors', 'Clinical Trial Identifier',
       'Title of study', 'Start Date', 'End Date', 'Sponsor Company',
       'Principal Investigator']]

In [35]:
# rename columns
df5.columns=['Company Name','Application No','IP Title','IP Year',
             'Inventors','Clinical Trial ID','Clinical Title',
            'Start Date', 'End Date', 'Sponsor', 'Investigator']

In [ ]:
df6=df5.iloc[:,:5]
dup_index=set(df6.index)-set(df6.drop_duplicates(keep='first').index)
df6.iloc[list(dup_index),1:]=np.NaN

In [ ]:
df7=df5.iloc[:,5:]
df7['a']=df5['Company Name']
dup_index=set(df7.index)-set(df7.drop_duplicates(keep='first').index)
df7.iloc[list(dup_index)]=np.NaN
df7=df7.drop('a',axis=1)

In [38]:
x=pd.concat([df6,df7],axis=1)

In [39]:
# add null columns as per template
for i in set(df.columns)-set(df5.columns):
    x[i]=np.nan
    
# rearrange columns as per template
x=x[df.columns]

In [40]:
y=x.groupby('Company Name').count()[['Application No','Clinical Trial ID']]
l1,l2=[],[]
for i in y['Application No']:
    if(i==0):
        l1.append('N')
    else:
        l1.append('Y')
for i in y['Clinical Trial ID']:
    if(i==0):
        l2.append('N')
    else:
        l2.append('Y')

In [41]:
y['Patent (Y/N)']=l1
y['Clinical Trials (Y/N)']=l2

In [42]:
y=y[['Patent (Y/N)','Application No','Clinical Trials (Y/N)','Clinical Trial ID']]
y.columns=['Patent (Y/N)','Value','Clinical Trials (Y/N)','Value']

In [43]:
y=y.sort_values(by=['Patent (Y/N)', 'Clinical Trials (Y/N)'])
ordered_classes = list(y.index)
df_list = []
for i in ordered_classes:
    df_list.append(x[x['Company Name']==i])
ordered_df = pd.concat(df_list)

In [44]:
# save to excel
name='/BiosectRx TEMPLATE-'+str(today)+'.xlsx'
writer = pd.ExcelWriter(directory+name, engine='xlsxwriter')
y.to_excel(writer, sheet_name='Check',index=True)
x.to_excel(writer, sheet_name='Harvest',index=False)
writer.save()